In [ ]:
IMPORTANT_KEYPOINTS= [
    "nose",
    "left_eye_inner",
    "left_eye",
    "left_eye_outer",
    "right_eye_inner",
    "right_eye",
    "right_eye_outer",
    "left_ear",
    "right_ear",
    "mouth_left",
    "mouth_right",
    "left_shoulder",
    "right_shoulder",
    "left_elbow",
    "right_elbow",
    "left_wrist",
    "right_wrist",
    "left_pinky",
    "right_pinky",
    "left_index",
    "right_index",
    "left_thumb",
    "right_thumb",
    "left_hip",
    "right_hip",
    "left_knee",
    "right_knee",
    "left_ankle",
    "right_ankle",
    "left_heel",
    "right_heel",
    "left_foot_index",
    "right_foot_index",
]

# Trích xuất Keypoints từ ảnh

Notebook này có nhiệm vụ:
1. Tiền xử lý ảnh từ thư mục `/data/train`
2. Trích xuất các keypoints sử dụng MediaPipe
3. Lưu các keypoints vào file CSV trong thư mục mới `/data_cleaned`

In [ ]:
# Import các thư viện cần thiết
import os
import sys
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm  # Sử dụng tqdm thông thường thay vì tqdm.notebook
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Thêm đường dẫn đến thư mục chứa các module tự định nghĩa
sys.path.append(str(Path('d:/GiaoPhan_Workspace/sem_6/pbl5/baby_posture_analysis/app')))

# Import các module tự định nghĩa
from utils.image_helper import Image_Helper, Image_Rotation_Helper
# Import helper mới cho việc trích xuất keypoints
# from keypoints_helper import KeypointsExtractorHelper
from utils.keypoints_helper import KeypointsExtractorHelper

In [ ]:
# Khởi tạo các đối tượng MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# Khởi tạo các đối tượng helper
image_helper = Image_Helper()
image_rotation_helper = Image_Rotation_Helper()
keypoints_helper = KeypointsExtractorHelper(IMPORTANT_KEYPOINTS, mp_pose)

In [ ]:
# Định nghĩa lớp KeypointExtractor
class KeypointExtractor:
    def __init__(self, mp_pose, important_keypoints):
        """
        Khởi tạo đối tượng KeypointExtractor
        
        Args:
            mp_pose: Đối tượng mp.solutions.pose
            important_keypoints: Danh sách các keypoint quan trọng cần trích xuất
        """
        self.mp_pose = mp_pose
        self.important_keypoints = important_keypoints
        self.pose = self.mp_pose.Pose(
            static_image_mode=True,
            model_complexity=2,
            enable_segmentation=True,
            min_detection_confidence=0.5
        )
        self.image_helper = Image_Helper()
        # Sử dụng KeypointsExtractorHelper thay vì PoseScalerHelper
        self.keypoints_helper = KeypointsExtractorHelper(important_keypoints, mp_pose)
        self.image_rotation_helper = Image_Rotation_Helper()
    
    def preprocess_image(self, image_path):
        """
        Tiền xử lý ảnh từ đường dẫn
        
        Args:
            image_path: Đường dẫn đến file ảnh
            
        Returns:
            image_rgb: Ảnh đã được tiền xử lý
        """
        try:
            # Đọc ảnh
            image = cv2.imread(str(image_path))
            
            # Kiểm tra xem ảnh có phải là ảnh grayscale không
            if len(image.shape) == 2 or image.shape[2] == 1:
                # Chuyển đổi grayscale sang RGB
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
            
            # Chuyển đổi ảnh sang định dạng vuông
            image_squared, new_size = self.image_helper.square_image(image)
            
            # Chuyển đổi ảnh sang RGB (MediaPipe yêu cầu RGB)
            image_rgb = cv2.cvtColor(image_squared, cv2.COLOR_BGR2RGB)
            
            return image_rgb
        except Exception as e:
            print(f"Lỗi tiền xử lý ảnh {image_path}: {str(e)}")
            return None
    
    def extract_keypoints(self, image):
        """
        Trích xuất keypoints từ ảnh
        
        Args:
            image: Ảnh cần trích xuất keypoints
            
        Returns:
            df_keypoints: DataFrame chứa các keypoints đã được trích xuất
            results: Kết quả trích xuất từ MediaPipe
        """
        try:
            results = self.pose.process(image)
            
            # Kiểm tra xem có phát hiện được pose không
            if not results.pose_landmarks:
                return None, None
            
            # Chuẩn hóa hướng của pose
            results.pose_landmarks = self.image_rotation_helper.rotate_image_baby(results.pose_landmarks)
            
            # Trích xuất và chuẩn hóa các keypoints sử dụng helper mới
            df_keypoints = self.keypoints_helper.process_keypoints(results.pose_landmarks.landmark)
            
            return df_keypoints, results
        except Exception as e:
            print(f"Lỗi trích xuất keypoints: {str(e)}")
            return None, None
    
    def process_image(self, image_path, label=None):
        """
        Xử lý ảnh và trích xuất keypoints
        
        Args:
            image_path: Đường dẫn đến file ảnh
            label: Nhãn của ảnh (tên lớp)
            
        Returns:
            df_keypoints: DataFrame chứa các keypoints đã được trích xuất
        """
        # Tiền xử lý ảnh
        image = self.preprocess_image(image_path)
        
        if image is None:
            return None
        
        # Trích xuất keypoints
        df_keypoints, results = self.extract_keypoints(image)
        
        if df_keypoints is None:
            return None
        
        # Thêm nhãn vào kết quả nếu có
        if label is not None:
            df_keypoints['label'] = label
            
        return df_keypoints
    
    def visualize_pose(self, image, results):
        """
        Hiển thị pose đã được phát hiện trên ảnh
        
        Args:
            image: Ảnh gốc
            results: Kết quả từ MediaPipe Pose
            
        Returns:
            annotated_image: Ảnh với pose được chú thích
        """
        if results is None or not results.pose_landmarks:
            return image
        
        # Copy ảnh để vẽ trên đó
        annotated_image = image.copy()
        
        # Vẽ landmarks
        mp_drawing.draw_landmarks(
            annotated_image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        
        return annotated_image

In [ ]:
# Thiết lập đường dẫn
root_path = Path('d:/GiaoPhan_Workspace/sem_6/pbl5/baby_posture_analysis')
data_source_path = root_path / 'data' / 'train'
data_dest_path = root_path / 'data_cleaned'

# Kiểm tra và tạo thư mục đích nếu chưa tồn tại
if not data_dest_path.exists():
    os.makedirs(data_dest_path)
    print(f"Đã tạo thư mục {data_dest_path}")
else:
    print(f"Thư mục {data_dest_path} đã tồn tại")

# Đường dẫn đến file CSV kết quả
csv_output_path = data_dest_path / 'keypoints_extract.csv'

In [ ]:
# Khởi tạo đối tượng KeypointExtractor
extractor = KeypointExtractor(mp_pose, IMPORTANT_KEYPOINTS)

# Danh sách các lớp
classes = ['nam_nghieng', 'nam_ngua', 'nam_sap']

# Danh sách DataFrame để lưu trữ kết quả
dfs = []

# Hiển thị một số ảnh mẫu
plt.figure(figsize=(15, 10))

# Biến đếm tổng số ảnh và số ảnh được xử lý thành công
total_images = 0
successful_images = 0

# Duyệt qua từng lớp và xử lý các ảnh
for idx, class_name in enumerate(classes):
    class_path = data_source_path / class_name
    image_files = list(class_path.glob('*.jpg')) + list(class_path.glob('*.png')) + list(class_path.glob('*.jpeg'))
    
    print(f"Xử lý {len(image_files)} ảnh trong lớp '{class_name}'")
    total_images += len(image_files)
    
    # Lựa chọn một ảnh mẫu để hiển thị
    if image_files:
        sample_image_path = image_files[0]  # Lấy ảnh đầu tiên làm mẫu
        sample_image = extractor.preprocess_image(sample_image_path)
        
        if sample_image is not None:
            df_keypoints, results = extractor.extract_keypoints(sample_image)
            
            if results is not None:
                # Vẽ pose lên ảnh mẫu
                annotated_image = extractor.visualize_pose(sample_image, results)
                
                # Hiển thị ảnh mẫu
                plt.subplot(1, 3, idx+1)
                plt.imshow(annotated_image)
                plt.title(f'Mẫu lớp {class_name}')
                plt.axis('off')
    
    # Xử lý tất cả các ảnh trong lớp
    for image_path in tqdm(image_files, desc=f"Lớp {class_name}"):
        try:
            # Trích xuất keypoints từ ảnh
            df_keypoints = extractor.process_image(image_path, class_name)
            
            if df_keypoints is not None:
                dfs.append(df_keypoints)
                successful_images += 1
        except Exception as e:
            print(f"Lỗi xử lý file {image_path}: {str(e)}")

plt.tight_layout()
plt.show()

# Hiển thị thông tin về quá trình xử lý
print(f"\nTổng số ảnh: {total_images}")
print(f"Số ảnh xử lý thành công: {successful_images}")
print(f"Tỉ lệ thành công: {successful_images/total_images*100:.2f}%")

In [ ]:
# Ghép tất cả các DataFrame lại với nhau
if dfs:
    final_df = pd.concat(dfs, ignore_index=True)
    
    # Lưu kết quả vào file CSV
    final_df.to_csv(csv_output_path, index=False)
    
    print(f"Đã lưu kết quả vào file {csv_output_path}")
    
    # Hiển thị thông tin về dữ liệu đã trích xuất
    print(f"\nThông tin dữ liệu đã trích xuất:")
    print(f"Kích thước: {final_df.shape}")
    print(f"Số lượng mẫu theo lớp:")
    print(final_df['label'].value_counts())
    
    # Hiển thị một số dòng đầu tiên
    print("\nXem một số dòng đầu tiên:")
    display(final_df.head())
else:
    print("Không có dữ liệu nào được trích xuất!")

## Phân tích phân phối keypoints

Visualize phân phối của các keypoints chính để kiểm tra dữ liệu

In [ ]:
if 'final_df' in locals() and not final_df.empty:
    # Chọn một số keypoints quan trọng để phân tích
    key_points = ['nose', 'left_shoulder', 'right_shoulder', 'left_hip', 'right_hip']
    
    # Hiển thị phân phối các keypoints theo lớp
    plt.figure(figsize=(15, 10))
    
    for i, point in enumerate(key_points):
        plt.subplot(2, 3, i+1)
        
        for label in final_df['label'].unique():
            subset = final_df[final_df['label'] == label]
            plt.scatter(subset[f'{point}_x'], subset[f'{point}_y'], alpha=0.7, label=label)
            
        plt.title(f'Phân bố {point}')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.grid(True, alpha=0.3)
        plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Hiển thị thống kê về dữ liệu
    print("\nThống kê về dữ liệu:")
    print(final_df.describe())
else:
    print("Không có dữ liệu để phân tích!")

## Kiểm tra dữ liệu bị thiếu hoặc outliers

Phân tích dữ liệu để tìm các giá trị bị thiếu hoặc outliers

In [ ]:
if 'final_df' in locals() and not final_df.empty:
    # Kiểm tra giá trị bị thiếu
    missing_values = final_df.isnull().sum()
    print("Các giá trị bị thiếu:")
    print(missing_values[missing_values > 0])
    
    # Kiểm tra outliers bằng cách vẽ boxplot cho một số keypoints quan trọng
    key_points = ['nose', 'left_shoulder', 'right_shoulder', 'left_hip', 'right_hip']
    axes = ['x', 'y']
    
    plt.figure(figsize=(15, 10))
    
    for i, point in enumerate(key_points):
        for j, axis in enumerate(axes):
            plt.subplot(len(key_points), len(axes), i*len(axes) + j + 1)
            
            col_name = f'{point}_{axis}'
            sns.boxplot(x='label', y=col_name, data=final_df)
            
            plt.title(f'{col_name} by Class')
            plt.xticks(rotation=45)
            plt.tight_layout()
    
    plt.tight_layout()
    plt.show()
else:
    print("Không có dữ liệu để kiểm tra!")

In [ ]:
# Import thư viện seaborn cho boxplot
import seaborn as sns

if 'final_df' in locals() and not final_df.empty:
    # Kiểm tra có bất kỳ outliers nào không bằng cách vẽ boxplot
    plt.figure(figsize=(20, 12))
    
    # Lựa chọn một số cột đại diện để phân tích
    sample_columns = [col for col in final_df.columns if ('_x' in col or '_y' in col) and not col.startswith('right_')][:10]
    
    for i, col in enumerate(sample_columns):
        plt.subplot(2, 5, i+1)
        sns.boxplot(x='label', y=col, data=final_df)
        plt.title(col)
        plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Xem phân phối tổng quát của dữ liệu
    print("\nPhân phối của dữ liệu theo lớp:")
    class_distribution = final_df['label'].value_counts()
    print(class_distribution)
    
    # Vẽ biểu đồ phân phối
    plt.figure(figsize=(10, 6))
    class_distribution.plot(kind='bar')
    plt.title('Số lượng mẫu theo lớp')
    plt.xlabel('Lớp')
    plt.ylabel('Số lượng')
    plt.xticks(rotation=45)
    
    for i, v in enumerate(class_distribution):
        plt.text(i, v + 0.5, str(v), ha='center')
    
    plt.tight_layout()
    plt.show()
else:
    print("Không có dữ liệu để phân tích!")

## Tóm tắt

Đã hoàn thành việc trích xuất keypoints từ các ảnh trong thư mục `/data/train` và lưu kết quả vào file `keypoints_extract.csv` trong thư mục `/data_cleaned`.

Kết quả:
- Số ảnh được xử lý thành công được ghi nhận ở trên
- Dữ liệu đã được chuẩn hóa và lưu vào file CSV
- Đã phân tích phân phối dữ liệu để kiểm tra chất lượng

Các keypoints được trích xuất từ mỗi ảnh bao gồm 33 điểm đặc trưng với tọa độ x, y, z cho mỗi điểm.